In [23]:
from env import config
import requests
import json

In [2]:
MINDSDB_UN = config("MINDSDB_UN", default=None)
MINDSDB_PW = config("MINDSDB_PW", default=None)
assert MINDSDB_UN is not None
assert MINDSDB_PW is not None


MINDSDB_BASE_URL = "https://cloud.mindsdb.com/api"

In [3]:
def get_mindsdb_session():
    session = requests.Session()
    session.post('https://cloud.mindsdb.com/cloud/login', json={
        'email': MINDSDB_UN,
        'password': MINDSDB_PW
    })
    return session

In [4]:
def mindsdb_query(session, sql_query):
    endpoint = "/sql/query"
    url = f"{MINDSDB_BASE_URL}{endpoint}"
    headers = {"Content-Type": "application/json"}
    return session.post(url, json={"query": sql_query}, headers=headers)

In [13]:
def predict_query(session, 
                  flightDate="2022-04-21", 
                  startingAirport="LAX", 
                  isNonStop=1,
                  isBasicEconomy=0,
                  isRefundable=0, 
                  destinationAirport="JFK",
                  raw_request=False,
                  **kwargs,
                 ):
    sql_query = f"""
    SELECT m.flightDate as date, m.segmentsAirlineName as airline, m.isNonStop as nonStop, m.isBasicEconomy as basic, m.isRefundable as refundable, m.totalFare as price FROM mindsdb.flight_price_predictor AS m
    JOIN ai_travel_agent.flight_prices AS t
    WHERE t.flightDate >= "{flightDate}"
    AND t.startingAirport = "{startingAirport}"
    AND t.isBasicEconomy = "{isBasicEconomy}"
    AND t.isRefundable = "{isRefundable}"
    AND t.isNonStop = {isNonStop}
    AND t.destinationAirport = "{destinationAirport}"
    LIMIT 10;
    """
    response = mindsdb_query(session, sql_query)
    response.raise_for_status()
    if raw_request:
        return response
    data = response.json()
    columns = data.get("column_names")
    dataset = data.get('data')
    if dataset is None or data is None:
        return []
    web_ready_data = [dict(zip(columns, row)) for row in dataset]
    return web_ready_data

In [14]:
session = get_mindsdb_session()
query_response = predict_query(session, startingAirport="SFO", destinationAirport="BOS", raw_request=False)
session.close()

In [16]:
# data = query_response.json()
# data.keys()

In [17]:
print(query_response)

[{'date': '2022-04-21 00:00:00.000000', 'airline': 'American Airlines', 'nonStop': '1', 'basic': '0', 'refundable': '0', 'price': 404.39715167400135}, {'date': '2022-04-21 00:00:00.000000', 'airline': 'JetBlue Airways', 'nonStop': '1', 'basic': '0', 'refundable': '0', 'price': 354.99898339656704}, {'date': '2022-04-21 00:00:00.000000', 'airline': 'Delta', 'nonStop': '1', 'basic': '0', 'refundable': '0', 'price': 352.59628080569775}, {'date': '2022-04-21 00:00:00.000000', 'airline': 'United', 'nonStop': '1', 'basic': '0', 'refundable': '0', 'price': 347.99920213019533}, {'date': '2022-04-21 00:00:00.000000', 'airline': 'Alaska Airlines', 'nonStop': '1', 'basic': '0', 'refundable': '0', 'price': 426.2468688253074}, {'date': '2022-04-22 00:00:00.000000', 'airline': 'American Airlines', 'nonStop': '1', 'basic': '0', 'refundable': '0', 'price': 404.40001195883394}, {'date': '2022-04-23 00:00:00.000000', 'airline': 'American Airlines', 'nonStop': '1', 'basic': '0', 'refundable': '0', 'price'

In [18]:
# columns = data.get("column_names")
# dataset = data.get("data")
# columns

NameError: name 'data' is not defined

In [20]:
query_response

[{'date': '2022-04-21 00:00:00.000000',
  'airline': 'American Airlines',
  'nonStop': '1',
  'basic': '0',
  'refundable': '0',
  'price': 404.39715167400135},
 {'date': '2022-04-21 00:00:00.000000',
  'airline': 'JetBlue Airways',
  'nonStop': '1',
  'basic': '0',
  'refundable': '0',
  'price': 354.99898339656704},
 {'date': '2022-04-21 00:00:00.000000',
  'airline': 'Delta',
  'nonStop': '1',
  'basic': '0',
  'refundable': '0',
  'price': 352.59628080569775},
 {'date': '2022-04-21 00:00:00.000000',
  'airline': 'United',
  'nonStop': '1',
  'basic': '0',
  'refundable': '0',
  'price': 347.99920213019533},
 {'date': '2022-04-21 00:00:00.000000',
  'airline': 'Alaska Airlines',
  'nonStop': '1',
  'basic': '0',
  'refundable': '0',
  'price': 426.2468688253074},
 {'date': '2022-04-22 00:00:00.000000',
  'airline': 'American Airlines',
  'nonStop': '1',
  'basic': '0',
  'refundable': '0',
  'price': 404.40001195883394},
 {'date': '2022-04-23 00:00:00.000000',
  'airline': 'American

In [24]:
pref = dict(flightDate="2022-04-21", 
          startingAirport="LAX", 
          isNonStop=1,
          destinationAirport="JFK",)
options = query_response
rec_context = json.dumps({"options": options, "preferences": pref})

In [25]:
rec_context

'{"options": [{"date": "2022-04-21 00:00:00.000000", "airline": "American Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 404.39715167400135}, {"date": "2022-04-21 00:00:00.000000", "airline": "JetBlue Airways", "nonStop": "1", "basic": "0", "refundable": "0", "price": 354.99898339656704}, {"date": "2022-04-21 00:00:00.000000", "airline": "Delta", "nonStop": "1", "basic": "0", "refundable": "0", "price": 352.59628080569775}, {"date": "2022-04-21 00:00:00.000000", "airline": "United", "nonStop": "1", "basic": "0", "refundable": "0", "price": 347.99920213019533}, {"date": "2022-04-21 00:00:00.000000", "airline": "Alaska Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 426.2468688253074}, {"date": "2022-04-22 00:00:00.000000", "airline": "American Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 404.40001195883394}, {"date": "2022-04-23 00:00:00.000000", "airline": "American Airlines", "nonStop": "1", "basic": "0", "refundable":

In [44]:
rec_sql_query = f"""
    SELECT answer
    FROM ai_travel_agent
    WHERE question="Respond with JSON only, What is the best flight option?"
    AND context='{rec_context}';
    """

In [32]:
session = get_mindsdb_session()
rec_prediction_response = mindsdb_query(session, rec_sql_query)
session.close()

In [33]:
rec_prediction_response.json()

{'column_names': ['answer'],
 'context': {'db': 'mindsdb'},
 'data': [['The best flight option is with United Airlines for $347.999.']],
 'type': 'table'}

In [38]:
pref = dict(flightDate="2022-04-21", 
          startingAirport="LAX", 
          isNonStop=1,
          destinationAirport="JFK",)
query_response = predict_query(session, **pref)
options = query_response
rec_context = json.dumps({"options": options, "preferences": pref})
rec_context

'{"options": [{"date": "2022-04-21 00:00:00.000000", "airline": "United", "nonStop": "1", "basic": "0", "refundable": "0", "price": 481.75790609878334}, {"date": "2022-04-21 00:00:00.000000", "airline": "Delta", "nonStop": "1", "basic": "0", "refundable": "0", "price": 423.995500019981}, {"date": "2022-04-21 00:00:00.000000", "airline": "American Airlines", "nonStop": "1", "basic": "0", "refundable": "0", "price": 439.79413415008287}, {"date": "2022-04-21 00:00:00.000000", "airline": "JetBlue Airways", "nonStop": "1", "basic": "0", "refundable": "0", "price": 414.99701251022907}, {"date": "2022-04-22 00:00:00.000000", "airline": "JetBlue Airways", "nonStop": "1", "basic": "0", "refundable": "0", "price": 415.00261280897803}, {"date": "2022-04-23 00:00:00.000000", "airline": "JetBlue Airways", "nonStop": "1", "basic": "0", "refundable": "0", "price": 415.00261280897803}, {"date": "2022-04-24 00:00:00.000000", "airline": "JetBlue Airways", "nonStop": "1", "basic": "0", "refundable": "0",

In [45]:
session = get_mindsdb_session()
rec_prediction_response = mindsdb_query(session, rec_sql_query)
session.close()

In [46]:
rec_prediction_response.json()

{'column_names': ['answer'],
 'context': {'db': 'mindsdb'},
 'data': [['{"date": "2022-04-21 00:00:00.000000", "airline": "Delta", "nonStop": "1", "basic": "0", "refundable": "0", "price": 423.995500019981}']],
 'type': 'table'}

In [48]:
def recomended_flight(session, 
                  user_data = {}, 
                  forecast_data = [], 
                  question = "Respond with JSON only, What is the best flight option?", 
                  raw_request=False,
                  **kwargs):
    context = {"options": forecast_data, "preferences": user_data}
    context_data = json.dumps(context)
    sql_query = f"""
    SELECT answer
    FROM ai_travel_agent
    WHERE question='{question}'
    AND context='{context_data}';
    """
    response = mindsdb_query(session, sql_query)
    response.raise_for_status()
    if raw_request:
        return response
    data = response.json()
    dataset = data.get('data')
    if dataset is None or data is None:
        return []
    if isinstance(dataset, list):
        sub_dataset = dataset[0]
        if isinstance(sub_dataset, list):
            return sub_dataset[0]
        return sub_dataset
    return dataset

In [50]:
json.loads(recomended_flight(session, pref, query_response))

{'date': '2022-04-21 00:00:00.000000',
 'airline': 'Delta',
 'nonStop': '1',
 'basic': '0',
 'refundable': '0',
 'price': 423.995500019981}